# Day 5: Supply Stacks

[https://adventofcode.com/2022/day/5](https://adventofcode.com/2022/day/5)

## Description

### Part One

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked _crates_, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a _giant cargo crane_ capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her _which_ crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates _and_ the rearrangement procedure (your puzzle input). For example:

        [D]    
    [N] [C]    
    [Z] [M] [P]
     1   2   3 
    
    move 1 from 2 to 1
    move 3 from 1 to 3
    move 2 from 2 to 1
    move 1 from 1 to 2
    

In this example, there are three stacks of crates. Stack 1 contains two crates: crate `Z` is on the bottom, and crate `N` is on top. Stack 2 contains three crates; from bottom to top, they are crates `M`, `C`, and `D`. Finally, stack 3 contains a single crate, `P`.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

    [D]        
    [N] [C]    
    [Z] [M] [P]
     1   2   3 
    

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved _one at a time_, so the first crate to be moved (`D`) ends up below the second and third crates:

            [Z]
            [N]
        [C] [D]
        [M] [P]
     1   2   3
    

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved _one at a time_, crate `C` ends up below crate `M`:

            [Z]
            [N]
    [M]     [D]
    [C]     [P]
     1   2   3
    

Finally, one crate is moved from stack 1 to stack 2:

            [Z]
            [N]
            [D]
    [C] [M] [P]
     1   2   3
    

The Elves just need to know _which crate will end up on top of each stack_; in this example, the top crates are `C` in stack 1, `M` in stack 2, and `Z` in stack 3, so you should combine these together and give the Elves the message _`CMZ`_.

_After the rearrangement procedure completes, what crate ends up on top of each stack?_

In [8]:
from pathlib import Path
from typing import Generator
import re

TEST = """\
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2""".splitlines()

EXPECTED_1 = "CMZ"
EXPECTED_2 = "MCD"

DATA = (Path("./input") / "data5.txt").read_text().splitlines()


def parse(
    data: list[str], cols: int
) -> tuple[list[list[str]], list[tuple[int, int, int]]]:
    stacks = [[] for _ in range(cols)]
    moves = []
    data = iter(data)
    for line in data:
        if not line:
            break
        for stack, val in zip(stacks, line[1::4]):
            if val != " ":
                stack.insert(0, val)
    for stack in stacks:
        del stack[0]
    for line in data:
        groups = re.match("move (\d+) from (\d+) to (\d+)", line)
        if groups:
            moves.append(
                (
                    int(groups[1]),
                    int(groups[2]),
                    int(groups[3]),
                )
            )
    return stacks, moves


def score_1(data: list[str], cols: int) -> str:
    stacks, moves = parse(data, cols)
    for count, src, dest in moves:
        for i in range(count):
            stacks[dest - 1].append(stacks[src - 1].pop(-1))
    return "".join(stack[-1] for stack in stacks)


test_score = score_1(TEST, 3)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA, 9))


test CMZ
part 1 VGBBJCRMN


<div class="alert alert-info">This was messy. Manipulating a list of strings instead of lists of lists would have been much simpler. Also I should have either inserted a dummy stack or subtracted 1 from `src` and `dest` in the parsing as those `- 1` were going to catch me out in part 2. I'm sure I could have worked out the column count from the input but just hardwiring it seemed simpler.</div>

### Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a _<span title="It's way better than the old CrateMover 1006.">CrateMover 9001</span>_.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and _the ability to pick up and move multiple crates at once_.

Again considering the example above, the crates begin in the same configuration:

        [D]    
    [N] [C]    
    [Z] [M] [P]
     1   2   3 
    

Moving a single crate from stack 2 to stack 1 behaves the same as before:

    [D]        
    [N] [C]    
    [Z] [M] [P]
     1   2   3 
    

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates _stay in the same order_, resulting in this new configuration:

            [D]
            [N]
        [C] [Z]
        [M] [P]
     1   2   3
    

Next, as both crates are moved from stack 2 to stack 1, they _retain their order_ as well:

            [D]
            [N]
    [C]     [Z]
    [M]     [P]
     1   2   3
    

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate `C` that gets moved:

            [D]
            [N]
            [Z]
    [M] [C] [P]
     1   2   3
    

In this example, the CrateMover 9001 has put the crates in a totally different order: _`MCD`_.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. _After the rearrangement procedure completes, what crate ends up on top of each stack?_


In [9]:

def score_2(data: list[str], cols: int) -> str:
    stacks, moves = parse(data, cols)
    for count, src, dest in moves:
        tmp = stacks[src - 1][-count:]
        stacks[src - 1][-count:] = []
        stacks[dest - 1].extend(tmp)
    return "".join(stack[-1] for stack in stacks)


if EXPECTED_2 is not None:
    test_score = score_2(TEST, 3)
    print("test", test_score)
    assert test_score == EXPECTED_2
    print("part 2", score_2(DATA, 9))


test MCD
part 2 LBBVJBRMH


<div class="alert alert-info">I missed one of the `- 1` which meant staring at the code for a bit trying to spot the mistake. Otherwise if anything simpler than the first (though I could stick in a [::-1] to get rid of the loop in the first one and have pretty nearly identical code).</div>

<div class="alert alert-info"></div>